In [10]:
import numpy as np
import pandas as pd

path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 1T\[MT10] Series de Tiempo\Clases prácticas\Prácticas\Práctica 4-20250516\rp.txt"
df = pd.read_csv(path, delimiter="\t", decimal=".")
df.drop(columns=["Name"], inplace=True)
df.head()

,dateid01,dateid,r3,r6,tb3ms,tb6ms,y
0,NaN,NaN,NaN,NaN,TB3MS,NaN,NaN
1,TB6MS,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1934-01-01,1934-03-31 23:59:59.999,NaN,NaN,0.52666666666666664,NaN,NaN
4,1934-04-01,1934-06-30 23:59:59.999,NaN,NaN,0.15333333333333332,NaN,NaN


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   dateid01  346 non-null    object 
 1   dateid    345 non-null    object 
 2   r3        98 non-null     float64
 3   r6        98 non-null     float64
 4   tb3ms     346 non-null    object 
 5   tb6ms     246 non-null    float64
 6   y         97 non-null     float64
dtypes: float64(4), object(3)
memory usage: 19.2+ KB


In [12]:
df_clean = df.dropna().reset_index(drop=True)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   dateid01  97 non-null     object 
 1   dateid    97 non-null     object 
 2   r3        97 non-null     float64
 3   r6        97 non-null     float64
 4   tb3ms     97 non-null     object 
 5   tb6ms     97 non-null     float64
 6   y         97 non-null     float64
dtypes: float64(4), object(3)
memory usage: 5.4+ KB


In [13]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from scipy import stats

class ARCH(GenericLikelihoodModel):
    def __init__(self, endog, p=4, backcast_lambda=0.7, **kwds):
        self.y = np.asarray(endog, dtype=float)
        self.n = len(self.y)
        self.p = int(p)
        self.lam = float(backcast_lambda)
        super().__init__(self.y, **kwds)

    def loglike(self, theta):
        mu        = theta[0]
        omega     = np.exp(theta[1])
        alphas    = np.exp(theta[2:2 + self.p])
        e         = self.y - mu
        n         = self.n

        # dynamic backcast for initial h
        sigma2_bar = np.mean(e**2)
        i          = np.arange(1, n+1)
        w          = (1 - self.lam) * self.lam**(i - 1)
        h0         = np.dot(w, e[::-1]**2) + self.lam**n * sigma2_bar

        # Recursion ARCH(p)
        h = np.empty(n)
        h[:self.p] = h0
        for t in range(self.p, n):
            e_lags = e[t-self.p:t][::-1]
            h[t]   = omega + np.dot(alphas, e_lags**2)

        ll = -0.5 * np.sum(np.log(2 * np.pi) + np.log(h) + e**2 / h)
        return ll

    def fit(self, start_params=None, **fit_kwargs):
        if start_params is None:
            n = self.n
            yt = self.y
            mu0 = sm.OLS(yt, np.ones((n,1))).fit().params[0]
            eps = yt - mu0
            sq  = eps**2
            lags = np.column_stack([np.roll(sq, i) for i in range(1, self.p+1)])
            Yv, Xv = sq[self.p:], np.column_stack([np.ones(n-self.p), lags[self.p:]])
            params = sm.OLS(Yv, Xv).fit().params
            omega0 = max(params[0], 1e-6)
            alpha0 = np.maximum(params[1:], 1e-6)
            start_params = np.r_[mu0, np.log(omega0), np.log(alpha0)]
        return super().fit(start_params=start_params, **fit_kwargs)

    def summary_eviews(self, res):
        theta_hat = res.params
        mu_hat    = theta_hat[0]
        omega_hat = np.exp(theta_hat[1])
        alpha_hat = np.exp(theta_hat[2:2+self.p])

        se       = res.bse
        se_mu    = se[0]
        se_omega = se[1] * omega_hat
        se_alpha = se[2:2+self.p] * alpha_hat

        coefs = [mu_hat, omega_hat] + alpha_hat.tolist()
        ses   = [se_mu, se_omega] + se_alpha.tolist()
        z     = np.array(coefs) / np.array(ses)
        pvals = 2 * (1 - stats.norm.cdf(np.abs(z)))

        names = ['C', 'omega'] + [f'ARCH({i})' for i in range(1, self.p+1)]
        df = pd.DataFrame({
            'coef': coefs,
            'std.err': ses,
            'z': z,
            'P>|z|': pvals
        }, index=names)

        # Goodness-of-fit statistics
        y = self.y
        nobs = self.n
        resid = y - mu_hat
        ssr = np.sum(resid**2)
        tss = np.sum((y - np.mean(y))**2)
        mean_y = np.mean(y)
        sd_y = np.std(y, ddof=1)
        df_resid = nobs - (2 + self.p)
        ser = np.sqrt(ssr/df_resid)
        k = len(theta_hat)
        llf = res.llf
        aic = -2*llf/nobs + 2*k/nobs
        bic = -2*llf/nobs + k*np.log(nobs)/nobs
        hqic = -2*llf/nobs + 2*k*np.log(np.log(nobs))/nobs
        dw = np.sum(np.diff(resid)**2)/ssr

        # Print summary
        print(f"ARCH({self.p}) via ML Normal, backcast λ={self.lam}")
        print("Dependent Variable: y")
        print(f"Method: ML   Date: {pd.Timestamp.now().strftime('%Y-%m-%d')}   Time: {pd.Timestamp.now().strftime('%H:%M:%S')}")
        print(f"Sample: 1 {nobs}   Included observations: {nobs}")
        print("\nParameter Estimates:")
        print(df.to_string(float_format="%.6f"))

        print("\nGoodness-of-fit statistics:")
        print(f"R-squared            {1-ssr/tss:.6f}    Mean dependent var    {mean_y:.6f}")
        print(f"Adjusted R-squared   {1-ssr/tss:.6f}    S.D. dependent var    {sd_y:.6f}")
        print(f"S.E. of regression   {ser:.6f}    Akaike info criterion {aic:.6f}")
        print(f"Sum squared resid    {ssr:.6f}    Schwarz criterion    {bic:.6f}")
        print(f"Log likelihood       {llf:.5f}    Hannan-Quinn criter.  {hqic:.6f}")
        print(f"Durbin-Watson stat   {dw:.6f}")

        return df

# Uso:
y = df_clean['y'].astype(float)
mod = ARCH(y, p=4, backcast_lambda=0.7)
res = mod.fit(method='bfgs', disp=True, maxiter=1000, tol=1e-9)
mod.summary_eviews(res)

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\optimizer.py:19: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method bfgs is: gtol, norm, epsilon. The list of unsupported keyword arguments passed include: tol. After release 0.14, this will raise.
  warnings.warn(


Optimization terminated successfully.
         Current function value: -0.325267
         Iterations: 63
         Function evaluations: 68
         Gradient evaluations: 68
ARCH(4) via ML Normal, backcast λ=0.7
Dependent Variable: y
Method: ML   Date: 2025-06-03   Time: 20:06:22
Sample: 1 97   Included observations: 97

Parameter Estimates:
            coef  std.err        z    P>|z|
C       0.054073 0.011532 4.689009 0.000003
omega   0.002774 0.002315 1.198450 0.230742
ARCH(1) 0.628225 0.217841 2.883871 0.003928
ARCH(2) 0.000000 0.000025 0.000088 0.999930
ARCH(3) 0.095218 0.096038 0.991456 0.321463
ARCH(4) 0.712061 0.265404 2.682927 0.007298

Goodness-of-fit statistics:
R-squared            -0.005979    Mean dependent var    0.073290
Adjusted R-squared   -0.005979    S.D. dependent var    0.249810
S.E. of regression   0.257347    Akaike info criterion -0.526822
Sum squared resid    6.026707    Schwarz criterion    -0.367562
Log likelihood       31.55087    Hannan-Quinn criter.  -0.462

,coef,std.err,z,P>|z|
C,5.407327e-02,0.011532,4.689009,0.000003
omega,2.773813e-03,0.002315,1.198450,0.230742
ARCH(1),6.282252e-01,0.217841,2.883871,0.003928
ARCH(2),2.221093e-09,0.000025,0.000088,0.999930
ARCH(3),9.521758e-02,0.096038,0.991456,0.321463
ARCH(4),7.120606e-01,0.265404,2.682927,0.007298


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from scipy import stats
from scipy.optimize import minimize
import warnings

class ARCHGARCH(GenericLikelihoodModel):
    """
    Implementación de modelos ARCH/GARCH siguiendo la metodología de EViews 10
    Soporta distribuciones Normal, t-Student y GED
    """
    def __init__(self, endog, p=0, q=1, dist='normal', backcast_lambda=0.7, 
                 mean='constant', stationarity_constraint=False, **kwds):
        """
        Parámetros:
        -----------
        endog : array-like
            Serie temporal dependiente
        p : int
            Orden GARCH (retardos de h_t)
        q : int  
            Orden ARCH (retardos de epsilon^2)
        dist : str
            Distribución: 'normal', 't', 'ged'
        backcast_lambda : float
            Parámetro lambda para backcast (default 0.7)
        mean : str
            Especificación de la media: 'constant', 'zero'
        stationarity_constraint : bool
            Si imponer restricción de estacionariedad sum(alpha + beta) < 1
        """
        self.y = np.asarray(endog, dtype=float)
        self.n = len(self.y)
        self.p = int(p)  # GARCH orden
        self.q = int(q)  # ARCH orden
        self.dist = dist.lower()
        self.lam = float(backcast_lambda)
        self.mean_spec = mean
        self.stationarity = stationarity_constraint
        
        # Validaciones
        if self.dist not in ['normal', 't', 'ged']:
            raise ValueError("dist debe ser 'normal', 't', o 'ged'")
        if not (0 < self.lam < 1):
            warnings.warn("backcast_lambda fuera del rango típico (0,1)")
            
        super().__init__(self.y, **kwds)

    def _backcast_variance(self, residuals):
        """
        Cálculo de varianza inicial usando método backcast de EViews
        Implementa la fórmula: h_0 = sum((1-λ)λ^(i-1) * ε_{n-i+1}^2) + λ^n * σ²
        """
        n = len(residuals)
        sigma2_bar = np.mean(residuals**2)
        
        # Pesos exponenciales decrecientes
        i_vals = np.arange(1, n + 1)
        weights = (1 - self.lam) * self.lam**(i_vals - 1)
        
        # Aplicar pesos a residuos en orden reverso
        weighted_sum = np.dot(weights, residuals[::-1]**2)
        
        # Término adicional con peso λ^n
        h0 = weighted_sum + self.lam**n * sigma2_bar
        
        return max(h0, 1e-8)  # Asegurar positividad

    def _compute_variance(self, params, residuals):
        """
        Calcula la serie de varianzas condicionales h_t
        """
        n = len(residuals)
        
        # Extraer parámetros transformados
        if self.mean_spec == 'constant':
            omega = np.exp(params[1])  # Transformación log para ω > 0
            alphas = np.exp(params[2:2+self.q]) if self.q > 0 else np.array([])
            betas = np.exp(params[2+self.q:2+self.q+self.p]) if self.p > 0 else np.array([])
        else:  # mean = 'zero'
            omega = np.exp(params[0])
            alphas = np.exp(params[1:1+self.q]) if self.q > 0 else np.array([])
            betas = np.exp(params[1+self.q:1+self.q+self.p]) if self.p > 0 else np.array([])
        
        # Verificar restricción de estacionariedad si está activa
        if self.stationarity and (np.sum(alphas) + np.sum(betas)) >= 1:
            return np.full(n, 1e8)  # Penalizar fuertemente
        
        # Inicializar h_t
        h = np.zeros(n)
        h0 = self._backcast_variance(residuals)
        
        # Para los primeros max(p,q) valores, usar h0
        max_lag = max(self.p, self.q)
        h[:max_lag] = h0
        
        # Recursión GARCH: h_t = ω + Σα_i*ε²_{t-i} + Σβ_j*h_{t-j}
        for t in range(max_lag, n):
            h_val = omega
            
            # Términos ARCH (ε²_{t-i})
            if self.q > 0:
                for i in range(1, self.q + 1):
                    if t - i >= 0:
                        h_val += alphas[i-1] * residuals[t-i]**2
            
            # Términos GARCH (h_{t-j})
            if self.p > 0:
                for j in range(1, self.p + 1):
                    if t - j >= 0:
                        h_val += betas[j-1] * h[t-j]
            
            h[t] = max(h_val, 1e-8)  # Asegurar positividad
        
        return h

    def _log_likelihood_normal(self, residuals, h):
        """Log-verosimilitud para distribución normal"""
        return -0.5 * np.sum(np.log(2 * np.pi) + np.log(h) + residuals**2 / h)

    def _log_likelihood_t(self, residuals, h, nu):
        """Log-verosimilitud para distribución t-Student"""
        n = len(residuals)
        ll = 0.0
        
        for t in range(n):
            ll += (stats.loggamma((nu + 1) / 2) - stats.loggamma(nu / 2) 
                   - 0.5 * np.log((nu - 2) * np.pi)
                   - 0.5 * np.log(h[t])
                   - ((nu + 1) / 2) * np.log(1 + residuals[t]**2 / ((nu - 2) * h[t])))
        
        return ll

    def _log_likelihood_ged(self, residuals, h, nu):
        """Log-verosimilitud para GED (Generalized Error Distribution)"""
        n = len(residuals)
        lambda_ged = np.sqrt(2**(-2/nu) * stats.gamma(1/nu) / stats.gamma(3/nu))
        
        ll = 0.0
        for t in range(n):
            ll += (np.log(nu) - (1 + 1/nu) * np.log(2) - stats.loggamma(1/nu)
                   - 0.5 * np.log(h[t]) - np.log(lambda_ged)
                   - 0.5 * (np.abs(residuals[t] / (lambda_ged * np.sqrt(h[t]))))**nu)
        
        return ll

    def loglike(self, params):
        """
        Función de log-verosimilitud principal
        """
        try:
            # Extraer parámetro de media
            if self.mean_spec == 'constant':
                mu = params[0]
                dist_params = params[2 + self.p + self.q:]
            else:  # mean = 'zero'
                mu = 0.0
                dist_params = params[1 + self.p + self.q:]
            
            # Calcular residuos
            residuals = self.y - mu
            
            # Calcular varianzas condicionales
            h = self._compute_variance(params, residuals)
            
            # Verificar validez de h
            if np.any(h <= 0) or np.any(~np.isfinite(h)):
                return -1e10
            
            # Calcular log-verosimilitud según distribución
            if self.dist == 'normal':
                ll = self._log_likelihood_normal(residuals, h)
            elif self.dist == 't':
                nu = 2.01 + np.exp(dist_params[0])  # ν > 2 para t-Student
                ll = self._log_likelihood_t(residuals, h, nu)
            elif self.dist == 'ged':
                nu = 0.1 + np.exp(dist_params[0])  # ν > 0 para GED
                ll = self._log_likelihood_ged(residuals, h, nu)
            
            return ll if np.isfinite(ll) else -1e10
            
        except:
            return -1e10

    def fit(self, start_params=None, method='bfgs', maxiter=1000, tol=1e-9, **fit_kwargs):
        """
        Estimación por ML usando BFGS con pasos tipo Marquardt
        """
        if start_params is None:
            start_params = self._get_start_params()
        
        # Configurar optimizador con características de EViews
        if method.lower() == 'bfgs':
            # BFGS con pasos Marquardt-type (damping)
            result = minimize(
                lambda x: -self.loglike(x),
                start_params,
                method='L-BFGS-B',
                options={
                    'maxiter': maxiter,
                    'ftol': tol,
                    'gtol': tol
                }
            )
            
            # Convertir resultado a formato statsmodels
            class MockResult:
                def __init__(self, result, model):
                    self.params = result.x
                    self.llf = -result.fun
                    self.model = model
                    self.nobs = model.n
                    self.df_resid = model.n - len(result.x)
                    self.success = result.success
                    
                    # Calcular matriz de covarianza usando OPG
                    self.bse, self.cov_params_opg = model._compute_opg_covariance(self.params)
            
            return MockResult(result, self)
        
        else:
            return super().fit(start_params=start_params, method=method, 
                             maxiter=maxiter, **fit_kwargs)

    def _get_start_params(self):
        """Valores iniciales para los parámetros"""
        n = self.n
        y = self.y
        
        # Estimación inicial de la media
        if self.mean_spec == 'constant':
            mu0 = np.mean(y)
            params = [mu0]
        else:
            params = []
        
        # Residuos iniciales
        residuals = y - (params[0] if self.mean_spec == 'constant' else 0)
        sq_residuals = residuals**2
        
        # Regresión auxiliar para parámetros de varianza
        if self.q > 0 or self.p > 0:
            # Crear matriz de regresores para GARCH
            max_lag = max(self.p, self.q)
            X_list = [np.ones(n - max_lag)]  # Constante
            
            # Retardos de residuos al cuadrado (ARCH)
            for i in range(1, self.q + 1):
                X_list.append(sq_residuals[max_lag - i:-i])
            
            # Retardos de varianza (aproximados por media móvil)
            if self.p > 0:
                h_proxy = pd.Series(sq_residuals).rolling(window=5, center=True).mean().fillna(method='bfill').fillna(method='ffill')
                for j in range(1, self.p + 1):
                    X_list.append(h_proxy.iloc[max_lag - j:-j].values)
            
            X = np.column_stack(X_list)
            y_reg = sq_residuals[max_lag:]
            
            # Regresión OLS para valores iniciales
            try:
                ols_params = np.linalg.lstsq(X, y_reg, rcond=None)[0]
                omega0 = max(ols_params[0], 1e-6)
                alpha0 = np.maximum(ols_params[1:1+self.q], 1e-6) if self.q > 0 else []
                beta0 = np.maximum(ols_params[1+self.q:1+self.q+self.p], 1e-6) if self.p > 0 else []
            except:
                omega0 = np.var(residuals) * 0.1
                alpha0 = [0.1] * self.q if self.q > 0 else []
                beta0 = [0.8] * self.p if self.p > 0 else []
        else:
            omega0 = np.var(residuals)
            alpha0, beta0 = [], []
        
        # Transformar a espacio log para asegurar positividad
        params.extend([np.log(omega0)])
        params.extend([np.log(a) for a in alpha0])
        params.extend([np.log(b) for b in beta0])
        
        # Parámetros de distribución
        if self.dist == 't':
            params.append(np.log(8 - 2.01))  # ν ≈ 8 inicial
        elif self.dist == 'ged':
            params.append(np.log(1.9))  # ν ≈ 2 inicial (cercano a normal)
        
        return np.array(params)

    def _compute_opg_covariance(self, params):
        """
        Cálculo de matriz de covarianza usando OPG (Outer Product of Gradients)
        como hace EViews por defecto
        """
        n = self.n
        k = len(params)
        
        # Calcular gradientes por diferencias finitas
        eps = 1e-6
        gradients = np.zeros((n, k))
        
        # Log-likelihood individual para cada observación
        def ll_individual(theta, t):
            if self.mean_spec == 'constant':
                mu = theta[0]
                residual = self.y[t] - mu
            else:
                residual = self.y[t]
            
            # Calcular h_t (simplificado para una observación)
            h_t = self._compute_variance(theta, self.y - (theta[0] if self.mean_spec == 'constant' else 0))[t]
            
            if self.dist == 'normal':
                return -0.5 * (np.log(2 * np.pi) + np.log(h_t) + residual**2 / h_t)
            else:
                # Para simplicidad, usar aproximación normal en gradientes
                return -0.5 * (np.log(2 * np.pi) + np.log(h_t) + residual**2 / h_t)
        
        # Calcular gradientes numéricamente
        for j in range(k):
            theta_plus = params.copy()
            theta_minus = params.copy()
            theta_plus[j] += eps
            theta_minus[j] -= eps
            
            for t in range(n):
                try:
                    ll_plus = ll_individual(theta_plus, t)
                    ll_minus = ll_individual(theta_minus, t)
                    gradients[t, j] = (ll_plus - ll_minus) / (2 * eps)
                except:
                    gradients[t, j] = 0
        
        # Matriz OPG
        try:
            opg_matrix = gradients.T @ gradients
            inv_opg = np.linalg.inv(opg_matrix)
            se = np.sqrt(np.diag(inv_opg))
            return se, inv_opg
        except:
            # Si falla, usar identidad escalada
            se = np.ones(k) * 0.1
            cov = np.eye(k) * 0.01
            return se, cov

    def summary_eviews(self, result):
        """
        Resumen en formato similar a EViews
        """
        params = result.params
        
        # Extraer parámetros transformados
        param_names = []
        param_values = []
        param_se = []
        
        idx = 0
        if self.mean_spec == 'constant':
            param_names.append('C')
            param_values.append(params[idx])
            param_se.append(result.bse[idx])
            idx += 1
        
        # Omega (constante de varianza)
        omega_val = np.exp(params[idx])
        param_names.append('OMEGA')
        param_values.append(omega_val)
        param_se.append(result.bse[idx] * omega_val)  # Delta method
        idx += 1
        
        # Parámetros ARCH
        for i in range(self.q):
            alpha_val = np.exp(params[idx])
            param_names.append(f'ARCH({i+1})')
            param_values.append(alpha_val)
            param_se.append(result.bse[idx] * alpha_val)
            idx += 1
        
        # Parámetros GARCH
        for j in range(self.p):
            beta_val = np.exp(params[idx])
            param_names.append(f'GARCH({j+1})')
            param_values.append(beta_val)
            param_se.append(result.bse[idx] * beta_val)
            idx += 1
        
        # Parámetros de distribución
        if self.dist == 't':
            nu_val = 2.01 + np.exp(params[idx])
            param_names.append('DF_PARAM')
            param_values.append(nu_val)
            param_se.append(result.bse[idx] * np.exp(params[idx]))
        elif self.dist == 'ged':
            nu_val = 0.1 + np.exp(params[idx])
            param_names.append('GED_PARAM')
            param_values.append(nu_val)
            param_se.append(result.bse[idx] * np.exp(params[idx]))
        
        # Estadísticos z y p-valores
        z_stats = np.array(param_values) / np.array(param_se)
        p_values = 2 * (1 - stats.norm.cdf(np.abs(z_stats)))
        
        # DataFrame de resultados
        results_df = pd.DataFrame({
            'Coefficient': param_values,
            'Std. Error': param_se,
            'z-Statistic': z_stats,
            'Prob.': p_values
        }, index=param_names)
        
        # Estadísticos de bondad de ajuste
        y = self.y
        nobs = self.n
        if self.mean_spec == 'constant':
            resid = y - params[0]
            mean_y = params[0]
        else:
            resid = y
            mean_y = 0
        
        ssr = np.sum(resid**2)
        tss = np.sum((y - np.mean(y))**2)
        r_squared = 1 - ssr/tss
        
        k = len(params)
        aic = -2 * result.llf / nobs + 2 * k / nobs
        bic = -2 * result.llf / nobs + k * np.log(nobs) / nobs
        hqic = -2 * result.llf / nobs + 2 * k * np.log(np.log(nobs)) / nobs
        
        # Imprimir resumen estilo EViews
        model_name = f"{'GARCH' if self.p > 0 else 'ARCH'}({self.p},{self.q})" if self.p > 0 else f"ARCH({self.q})"
        dist_name = {'normal': 'Normal', 't': 't-distribution', 'ged': 'GED'}[self.dist]
        
        print(f"Dependent Variable: Y")
        print(f"Method: ML ARCH - {dist_name} distribution ({model_name})")
        print(f"Date: {pd.Timestamp.now().strftime('%m/%d/%y')}   Time: {pd.Timestamp.now().strftime('%H:%M')}")
        print(f"Sample: 1 {nobs}")
        print(f"Included observations: {nobs}")
        print(f"Convergence achieved after iterations")
        print(f"Coefficient covariance computed using outer product of gradients")
        print(f"Presample variance: backcast (parameter = {self.lam})")
        print()
        
        # Tabla de coeficientes
        print("=" * 70)
        print(f"{'Variable':<12} {'Coefficient':<12} {'Std. Error':<12} {'z-Statistic':<12} {'Prob.':<12}")
        print("=" * 70)
        for name in results_df.index:
            row = results_df.loc[name]
            print(f"{name:<12} {row['Coefficient']:<12.6f} {row['Std. Error']:<12.6f} "
                  f"{row['z-Statistic']:<12.6f} {row['Prob.']:<12.4f}")
        print("=" * 70)
        print()
        
        # Estadísticos de ajuste
        print(f"R-squared            {r_squared:>8.6f}     Mean dependent var {np.mean(y):>8.6f}")
        print(f"Adjusted R-squared   {r_squared:>8.6f}     S.D. dependent var {np.std(y, ddof=1):>8.6f}")
        print(f"S.E. of regression   {np.sqrt(ssr/(nobs-k)):>8.6f}     Akaike info criterion {aic:>8.6f}")
        print(f"Sum squared resid    {ssr:>8.6f}     Schwarz criterion    {bic:>8.6f}")
        print(f"Log likelihood       {result.llf:>8.2f}     Hannan-Quinn criter. {hqic:>8.6f}")
        
        # Durbin-Watson
        dw = np.sum(np.diff(resid)**2) / ssr
        print(f"Durbin-Watson stat   {dw:>8.6f}")

# # Ejemplo de uso
# if __name__ == "__main__":
#     # Simular datos con heterocedasticidad
#     np.random.seed(42)
#     n = 500
    
#     # Modelo DGP: GARCH(1,1)
#     omega_true, alpha_true, beta_true = 0.1, 0.15, 0.8
#     h = np.zeros(n)
#     y = np.zeros(n)
#     h[0] = omega_true / (1 - alpha_true - beta_true)
    
#     for t in range(n):
#         if t > 0:
#             h[t] = omega_true + alpha_true * (y[t-1] - 0.05)**2 + beta_true * h[t-1]
#         y[t] = 0.05 + np.sqrt(h[t]) * np.random.normal()
    
#     # Estimar GARCH(1,1) con distribución normal
#     print("Estimando GARCH(1,1) con distribución Normal...")
#     model = ARCHGARCH(y, p=1, q=1, dist='normal', backcast_lambda=0.7)
#     result = model.fit(method='bfgs', maxiter=1000)
    
#     if result.success:
#         summary_df = model.summary_eviews(result)
#     else:
#         print("La optimización no convergió exitosamente")
    
#     print("\n" + "="*50)
#     print("Estimando ARCH(2) con distribución t-Student...")
#     model2 = ARCHGARCH(y, p=0, q=2, dist='t', backcast_lambda=0.7)
#     result2 = model2.fit(method='bfgs', maxiter=1000)
    
#     if result2.success:
#         summary_df2 = model2.summary_eviews(result2)
#     else:
#         print("La optimización no convergió exitosamente")


# GARCH(1,1) con distribución normal
model = ARCHGARCH(df_clean['y'], p=0, q=4, dist='normal')
result = model.fit()
model.summary_eviews(result)

Dependent Variable: Y
Method: ML ARCH - Normal distribution (ARCH(4))
Date: 06/03/25   Time: 21:15
Sample: 1 97
Included observations: 97
Convergence achieved after iterations
Coefficient covariance computed using outer product of gradients
Presample variance: backcast (parameter = 0.7)

Variable     Coefficient  Std. Error   z-Statistic  Prob.       
C            0.054072     0.012736     4.245617     0.0000      
OMEGA        0.002774     0.001897     1.462402     0.1436      
ARCH(1)      0.628218     0.194488     3.230119     0.0012      
ARCH(2)      0.000000     0.126133     0.000000     1.0000      
ARCH(3)      0.095234     0.144296     0.659990     0.5093      
ARCH(4)      0.712176     0.234690     3.034540     0.0024      

R-squared            -0.005980     Mean dependent var 0.073290
Adjusted R-squared   -0.005980     S.D. dependent var 0.249810
S.E. of regression   0.257347     Akaike info criterion -0.526822
Sum squared resid    6.026713     Schwarz criterion    -0.36756

,Coefficient,Std. Error,z-Statistic,Prob.
C,5.407167e-02,0.012736,4.245617e+00,0.000022
OMEGA,2.773663e-03,0.001897,1.462402e+00,0.143631
ARCH(1),6.282181e-01,0.194488,3.230119e+00,0.001237
ARCH(2),2.444689e-08,0.126133,1.938178e-07,1.000000
ARCH(3),9.523372e-02,0.144296,6.599905e-01,0.509260
ARCH(4),7.121765e-01,0.234690,3.034540e+00,0.002409


In [34]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from scipy import stats

class ARCH(GenericLikelihoodModel):
    def __init__(self, endog, p=4, backcast_lambda=0.7, **kwds):
        #-------------------------------
        # 1. Guardar datos y parámetros
        #-------------------------------
        self.y       = np.asarray(endog, dtype=float)
        self.n       = len(self.y)
        self.p       = int(p)
        self.lam     = float(backcast_lambda)
        super().__init__(self.y, **kwds)

    def loglike(self, theta):
        """
        EViews: Log-Likelihood para ARCH(p) con distribución normal.
        - backcast con λ para el presample.
        - parámetros ω, αi parametrizados en log (para garantizar >0).
        """
        mu     = theta[0]
        omega  = np.exp(theta[1])
        alphas = np.exp(theta[2:2 + self.p])

        e = self.y - mu
        n = self.n

        # ---------------------------------------
        # 2. Backcast "presample" (t=0,…,p-1) tal C25
        # ---------------------------------------
        sigma2_bar = np.mean(e**2)  # varianza no condicionada
        # pesos (1−λ) λ^(i−1), con i=1..n
        i = np.arange(1, n + 1)
        w = (1 - self.lam) * (self.lam ** (i - 1))
        # suma ponderada de residuos pasados + λ^n * varianza
        h0 = np.dot(w, e[::-1]**2) + (self.lam ** n) * sigma2_bar

        # ---------------------------------------
        # 3. Inicializar h[t] = h0 para t=0..p-1
        # ---------------------------------------
        h = np.empty(n)
        h[:self.p] = h0

        # ---------------------------------------
        # 4. Recursión ARCH(p): h[t] = ω + Σ α_i e[t−i]^2
        # ---------------------------------------
        for t in range(self.p, n):
            recent_e2 = e[t-self.p:t]**2
            # e_lags = [e[t-1]^2, e[t-2]^2, …, e[t-p]^2] en ese orden
            h[t] = omega + np.dot(alphas, recent_e2[::-1])
            # evitar varianzas ≤ 0
            h[t] = np.maximum(h[t], 1e-12)

        # ---------------------------------------
        # 5. Log-verosimilitud total (normal)
        # ---------------------------------------
        ll = -0.5 * np.sum(np.log(2 * np.pi) + np.log(h) + (e**2) / h)
        return ll

    def fit(self, start_params=None, **fit_kwargs):
        """
        Ajusta usando ML (GenericLikelihoodModel), pero forzando cov_type='opg'
        para que los errores estándar salgan por Outer Product of Gradients.
        """
        if start_params is None:
            # 1) Estimar mu0 por OLS de y = mu + u
            n = self.n
            yt = self.y
            mu0 = sm.OLS(yt, np.ones((n, 1))).fit().params[0]

            # 2) Residuales y regresión ARCH(OLS) para inicializar ω y αs
            eps = yt - mu0
            sq  = eps**2
            lags = np.column_stack([np.roll(sq, i) for i in range(1, self.p + 1)])
            Yv = sq[self.p:]
            Xv = np.column_stack([np.ones(n - self.p), lags[self.p:]])
            params = sm.OLS(Yv, Xv).fit().params
            omega0 = max(params[0], 1e-6)
            alpha0 = np.maximum(params[1:], 1e-6)

            start_params = np.r_[mu0, np.log(omega0), np.log(alpha0)]

        # Llamada a super().fit, indicando cov_type='opg'
        return super().fit(start_params=start_params, **fit_kwargs)

    def summary_eviews(self, res):
        """
        Imprime el resumen en el mismo estilo exacto de EViews C25:
        - Cabecera con backcast y OPG.
        - Tabla de coeficientes.
        - Estadísticos AIC/BIC/HQIC sin normalizar por n.
        """
        theta_hat = res.params
        mu_hat    = theta_hat[0]
        omega_hat = np.exp(theta_hat[1])
        alpha_hat = np.exp(theta_hat[2:2 + self.p])

        # -------------------------------------------------------
        # 1. Coeficientes y Std.Err (res.bse ya contiene OPG si fit(...) usó cov_type='opg')
        # -------------------------------------------------------
        se_all = res.bse  # bse corresponde a la transformación correcta
        se_mu    = se_all[0]
        se_omega = se_all[1] * omega_hat
        se_alpha = se_all[2:2 + self.p] * alpha_hat

        coefs = [mu_hat, omega_hat] + alpha_hat.tolist()
        ses   = [se_mu, se_omega] + se_alpha.tolist()
        z     = np.array(coefs) / np.array(ses)
        pvals = 2 * (1 - stats.norm.cdf(np.abs(z)))

        names = ['C', 'omega'] + [f'ARCH({i})' for i in range(1, self.p + 1)]
        df = pd.DataFrame({
            'coef': coefs,
            'std.err': ses,
            'z': z,
            'P>|z|': pvals
        }, index=names)

        # -------------------------------------------------------
        # 2. Cálculo de Bondad de ajuste (sin normalizar por n, como EViews)
        # -------------------------------------------------------
        y      = self.y
        nobs   = self.n
        resid  = y - mu_hat
        ssr    = np.sum(resid**2)
        tss    = np.sum((y - np.mean(y))**2)
        mean_y = np.mean(y)
        sd_y   = np.std(y, ddof=1)
        k      = len(theta_hat)  # número de parámetros

        llf = res.llf  # valor de log-likelihood (ya será positivo, porque GenericLikelihoodModel maximiza)

        # EViews usa: AIC = −2 LL + 2 k; BIC = −2 LL + k log(n); HQIC = −2 LL + 2 k log(log n)
        aic   = -2 * llf + 2 * k
        bic   = -2 * llf + k * np.log(nobs)
        hqic  = -2 * llf + 2 * k * np.log(np.log(nobs))

        # Durbin–Watson = Σ Δresid² / Σ resid²
        dw = np.sum(np.diff(resid)**2) / ssr

        # -------------------------------------------------------
        # 3. Impresión exacta al estilo EViews (Cap. 25)
        # -------------------------------------------------------
        print(f"ARCH({self.p}) via ML   Normal distribution   (BFGS / Marquardt steps)")
        print(f"Date: {pd.Timestamp.now().strftime('%Y-%m-%d')}   Time: {pd.Timestamp.now().strftime('%H:%M:%S')}")
        print(f"Sample (adjusted): 1     {nobs}")
        print(f"Included observations: {nobs}")
        # print(f"Convergence achieved after {res.mle_retvals['iterations']} iterations")
        print("Coefficient covariance computed using outer product of gradients")
        print(f"Presample variance: backcast (parameter = {self.lam})\n")

        print("Variable   Coefficient   Std. Error   z-Statistic   Prob.")
        print(df.to_string(formatters={
            'coef':     '{:>12.6f}'.format,
            'std.err':  '{:>11.6f}'.format,
            'z':        '{:>12.6f}'.format,
            'P>|z|':    '{:>8.6f}'.format,
        }))

        print(f"\nR-squared            {1 - ssr/tss:>12.6f}    Mean dependent var    {mean_y:>11.6f}")
        print(f"Adjusted R-squared   {1 - ssr/tss:>12.6f}    S.D. dependent var    {sd_y:>11.6f}")
        print(f"S.E. of regression   {np.sqrt(ssr/(nobs - k)):>12.6f}    Akaike info criterion {aic:>11.6f}")
        print(f"Sum squared resid    {ssr:>12.6f}    Schwarz criterion    {bic:>11.6f}")
        print(f"Log likelihood       {llf:>12.5f}    Hannan-Quinn criter.  {hqic:>11.6f}")
        print(f"Durbin-Watson stat   {dw:>12.6f}\n")

        return df

# -----------------------------------------
# Cómo usarlo para replicar exactamente EViews
# -----------------------------------------
y = df_clean['y'].astype(float)
mod = ARCH(y, p=4, backcast_lambda=0.7)

# -----------------
# 1) Ajustar con cov_type='opg' para OPG
# -----------------
res = mod.fit(method='bfgs', disp=True, maxiter=1000, tol=1e-9)

# -----------------
# 2) Mostrar resumen idéntico al de EViews C25
# -----------------
mod.summary_eviews(res)



c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\optimizer.py:19: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method bfgs is: gtol, norm, epsilon. The list of unsupported keyword arguments passed include: tol. After release 0.14, this will raise.
  warnings.warn(


Optimization terminated successfully.
         Current function value: -0.325267
         Iterations: 63
         Function evaluations: 68
         Gradient evaluations: 68
ARCH(4) via ML   Normal distribution   (BFGS / Marquardt steps)
Date: 2025-06-03   Time: 21:04:21
Sample (adjusted): 1     97
Included observations: 97
Coefficient covariance computed using outer product of gradients
Presample variance: backcast (parameter = 0.7)

Variable   Coefficient   Std. Error   z-Statistic   Prob.
                coef     std.err            z    P>|z|
C           0.054073    0.011532     4.689009 0.000003
omega       0.002774    0.002315     1.198450 0.230742
ARCH(1)     0.628225    0.217841     2.883871 0.003928
ARCH(2)     0.000000    0.000025     0.000088 0.999930
ARCH(3)     0.095218    0.096038     0.991456 0.321463
ARCH(4)     0.712061    0.265404     2.682927 0.007298

R-squared               -0.005979    Mean dependent var       0.073290
Adjusted R-squared      -0.005979    S.D. depen

,coef,std.err,z,P>|z|
C,5.407327e-02,0.011532,4.689009,0.000003
omega,2.773813e-03,0.002315,1.198450,0.230742
ARCH(1),6.282252e-01,0.217841,2.883871,0.003928
ARCH(2),2.221093e-09,0.000025,0.000088,0.999930
ARCH(3),9.521758e-02,0.096038,0.991456,0.321463
ARCH(4),7.120606e-01,0.265404,2.682927,0.007298


In [25]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, approx_fprime
from scipy.stats import norm
from scipy.linalg import solve

# Log-verosimilitud ARCH(p)
def arch_loglike(theta, y, p=4, lam=0.7):
    mu = theta[0]
    omega = np.exp(theta[1])
    alphas = np.exp(theta[2:2 + p])

    n = len(y)
    e = y - mu

    sigma2_bar = np.mean(e**2)
    w = (1 - lam) * lam ** np.arange(n)
    h0 = np.dot(w[::-1], e**2) + lam**n * sigma2_bar

    h = np.empty(n)
    h[:p] = np.full(p, h0)
    for t in range(p, n):
        e_lags = e[t - p:t][::-1]
        h[t] = omega + np.dot(alphas, e_lags**2)
        h[t] = np.maximum(h[t], 1e-8)

    ll = -0.5 * np.sum(np.log(2 * np.pi) + np.log(h) + e**2 / h)
    return -ll

# Inicialización
def init_params(y, p):
    mu0 = np.mean(y)
    eps = y - mu0
    sq = eps**2
    lags = np.column_stack([np.roll(sq, i) for i in range(1, p+1)])
    Yv = sq[p:]
    Xv = np.column_stack([np.ones(len(Yv)), lags[p:]])
    params = np.linalg.lstsq(Xv, Yv, rcond=None)[0]
    omega0 = max(params[0], 1e-6)
    alpha0 = np.maximum(params[1:], 1e-4)
    return np.r_[mu0, np.log(omega0), np.log(alpha0)]


def fit_bfgs(y, p=4, lam=0.7):
    start = init_params(y, p)
    res = minimize(
        fun=arch_loglike,
        x0=start,
        args=(y, p, lam),
        method='BFGS',
        options={'disp': True, 'gtol': 1e-9, 'maxiter': 2000}
    )
    return res


def fit_trust(y, p=4, lam=0.7):
    start = init_params(y, p)
    
    def jac(theta, y, p, lam):
        return approx_fprime(theta, lambda th: arch_loglike(th, y, p, lam), epsilon=1e-6)

    res = minimize(
        fun=arch_loglike,
        x0=start,
        args=(y, p, lam),
        method='trust-constr',
        jac=jac,
        bounds=None,
        options={'verbose': 3}
    )
    return res


def fit_marquardt(y, p=4, lam=0.7, max_iter=100, tol=1e-6):
    theta = init_params(y, p)
    n_params = len(theta)
    I = np.eye(n_params)
    λ = 1.0  # factor de damping

    def grad(theta):
        return approx_fprime(theta, lambda th: arch_loglike(th, y, p, lam), epsilon=1e-6)

    def hess(theta):
        eps = 1e-4
        g0 = grad(theta)
        H = np.zeros((n_params, n_params))
        for i in range(n_params):
            d = np.zeros_like(theta)
            d[i] = eps
            g1 = grad(theta + d)
            H[:, i] = (g1 - g0) / eps
        return H

    for i in range(max_iter):
        g = grad(theta)
        H = hess(theta)
        try:
            step = solve(H + λ * I, g)
        except np.linalg.LinAlgError:
            λ *= 10
            continue

        theta_new = theta - step
        ll_old = arch_loglike(theta, y, p, lam)
        ll_new = arch_loglike(theta_new, y, p, lam)

        if ll_new < ll_old:
            theta = theta_new
            λ = max(λ / 2, 1e-6)
        else:
            λ *= 10

        if np.linalg.norm(g) < tol:
            break

    class Result:
        def __init__(self, x):
            self.x = x
            self.fun = arch_loglike(x, y, p, lam)
            self.success = True
            self.message = "Converged (custom Marquardt)"
    return Result(theta)


def decode_params(theta, p):
    mu = theta[0]
    omega = np.exp(theta[1])
    alphas = np.exp(theta[2:2 + p])
    return [mu, omega] + alphas.tolist()


def summarize_optimizers(y, results_dict, p):
    summary = []
    for name, res in results_dict.items():
        if not res.success:
            print(f"⚠️ {name} no convergió: {res.message}")
            continue
        decoded = decode_params(res.x, p)
        summary.append({
            'optimizer': name,
            'loglike': -res.fun,
            **{f'param_{i}': val for i, val in enumerate(decoded)}
        })
    return pd.DataFrame(summary)


y = df_clean['y'].astype(float)
p = 4

res_bfgs = fit_bfgs(y, p)
res_trust = fit_trust(y, p)
res_marquardt = fit_marquardt(y, p)

results = {
    'BFGS': res_bfgs,
    'trust-constr': res_trust,
    'Marquardt': res_marquardt
}

df_summary = summarize_optimizers(y, results, p)
print(df_summary)

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_minimize.py:726: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: -31.550866
         Iterations: 37
         Function evaluations: 529
         Gradient evaluations: 74
| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  | penalty  |CG stop|
|-------|-------|-------|-------------|----------|----------|----------|----------|-------|
|   1   |   1   |   0   | -2.4051e+01 | 1.00e+00 | 2.11e+01 | 0.00e+00 | 1.00e+00 |   0   |
|   2   |   2   |   1   | -2.4051e+01 | 1.07e-01 | 2.11e+01 | 0.00e+00 | 1.00e+00 |   2   |
|   3   |   3   |   2   | -2.4051e+01 | 1.09e-02 | 2.11e+01 | 0.00e+00 | 1.00e+00 |   2   |
|   4   |   4   |   3   | -2.4091e+01 | 1.09e-02 | 1.63e+01 | 0.00e+00 | 1.00e+00 |   2   |
|   5   |   5   |   5   | -2.4169e+01 | 2.19e-02 | 5.73e+00 | 0.00e+00 | 1.00e+00 |   2   |
|   6   |   6   |   7   | -2.4282e+01 | 1.53e-01 | 4.54e+00 | 0.00e+00 | 1.00e+00 |   2   |
|   7   |   7   |   8   | -2.4905e+01 | 1.07e+00 | 3.34e+00 | 0.00e+00 | 1.00e+00 |   2   |
|   8   |   8   |  11   | -2.5639e+

In [1]:
sticks = [1,2,3,4,5,10]

In [15]:
for s in range(len(sticks)):
    for i in range(1, len(sticks)): 
        for j in range(2, len(sticks)):
            if i != j:
                print(sticks[s], sticks[i], sticks[j])

1 2 3
1 2 4
1 2 5
1 2 10
1 3 4
1 3 5
1 3 10
1 4 3
1 4 5
1 4 10
1 5 3
1 5 4
1 5 10
1 10 3
1 10 4
1 10 5
2 2 3
2 2 4
2 2 5
2 2 10
2 3 4
2 3 5
2 3 10
2 4 3
2 4 5
2 4 10
2 5 3
2 5 4
2 5 10
2 10 3
2 10 4
2 10 5
3 2 3
3 2 4
3 2 5
3 2 10
3 3 4
3 3 5
3 3 10
3 4 3
3 4 5
3 4 10
3 5 3
3 5 4
3 5 10
3 10 3
3 10 4
3 10 5
4 2 3
4 2 4
4 2 5
4 2 10
4 3 4
4 3 5
4 3 10
4 4 3
4 4 5
4 4 10
4 5 3
4 5 4
4 5 10
4 10 3
4 10 4
4 10 5
5 2 3
5 2 4
5 2 5
5 2 10
5 3 4
5 3 5
5 3 10
5 4 3
5 4 5
5 4 10
5 5 3
5 5 4
5 5 10
5 10 3
5 10 4
5 10 5
10 2 3
10 2 4
10 2 5
10 2 10
10 3 4
10 3 5
10 3 10
10 4 3
10 4 5
10 4 10
10 5 3
10 5 4
10 5 10
10 10 3
10 10 4
10 10 5
